# Smarter Prompting, Faster: Introducing Opik's Agent Optimizers

Doug Blank, Phd

* Slides are available at: [bit.ly/opik-optimizer-dsblank-slides](https://bit.ly/opik-optimizer-dsblank-slides)
* This notebook is available at: [bit.ly/opik-optimizer-dsblank](https://bit.ly/opik-optimizer-dsblank)

You will need:
1. A Google account, for running a Colab Notebook  - [google.com](https://google.com)
2. A Comet account, for seeing Opik visualizations (free!) - [comet.com](https://comet.com)
3. An OpenAI account, for using an LLM
[platform.openai.com/settings/organization/api-keys](https://platform.openai.com/settings/organization/api-keys)


## Setup

This pip-install takes about a minute.

In [2]:
%%capture
%pip install opik-optimizer

In [3]:
import opik_optimizer
opik_optimizer.__version__

'0.8.1'

In [4]:
import opik

# Configure Opik
opik.configure()

OPIK: Opik is already configured. You can check the settings by viewing the config file at /Users/jacquesverre/.opik.config


In [5]:
import os
import getpass
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

To save time (and money) durring this demonstration, we have capture the results of a previous run of all of these LLM interactions. Our goal is to make this not cost any money. However, we of course can guarantee that. **Use at your own risk**!

To capture the perviously cached results:

In [6]:
from opik_optimizer.demo.cache import get_litellm_cache

get_litellm_cache("opik-workshop")

Inserted 0 record(s) in litellm cache


## The Dataset

In these set of experiments, we are going to use the **HotPotQA** dataset. This dataset was designed to be difficult for regular LLMs to handle. This dataset is called a "**multi-hop**" dataset because answering the questions involves multiple reasoning steps and multiple tool calls, where the LLM needs to infer relationships, combine information, or draw conclusions based on the combined context.

Example:

> "What are the capitals of the states that border California?"

You'd need to find which states border California, and then lookup each state's capital.

The dataset has about 113,000 such crowd-sourced questions that are constructed to require the introductory paragraphs of two Wikipedia articles to answer.

[1] The name "HotPot" comes from the restaurant where the authors came up with the idea of the dataset.

In [7]:
import opik_optimizer

opik_dataset = opik_optimizer.datasets.hotpot_300()

Let's take a look at some dataset items:

In [8]:
rows = opik_dataset.get_items()
rows[0]

{'id': '0197044c-d782-7735-a8cf-bffd4d19838e',
 'question': 'Are Smyrnium and Nymania both types of plant?',
 'answer': 'yes'}

In [9]:
rows[1]

{'id': '0197044c-d781-7eb0-a717-6fd6b1415476',
 'question': 'That Darn Cat! and Never a Dull Moment were both produced by what studio?',
 'answer': 'Walt Disney Productions'}

## Opik Project

All LLM traces in Opik are saved in a "project". We'll put them all in the following project name:

In [10]:
project_name = "optimize-workshop-2025"

## The Metric

Choosing a good metric for optimization is tricky. For these examples, we'll pick one that will allow us to show improvement, and also provide a gradient of scores. In general though, this metric isn't the best for optimization runs.

We'll use "Edit Distance" AKA "Levenshtein Distance":

In [11]:
from opik.evaluation.metrics import LevenshteinRatio

metric = LevenshteinRatio(project_name=project_name)

The metric takes two things: the output of the LLM and the reference (correct answer).

In [12]:
metric.score("Hello", "Hello")

OPIK: Started logging traces to the "optimize-workshop-2025" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=0197406b-3128-787c-8d18-1d9138daee1a&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.


ScoreResult(name='levenshtein_ratio_metric', value=1.0, reason=None, metadata=None, scoring_failed=False)

In [13]:
metric.score("Hello!", "Hello")

ScoreResult(name='levenshtein_ratio_metric', value=0.9090909090909091, reason=None, metadata=None, scoring_failed=False)

The edit distance between "Hello!" and "Hello" is 1. Here is how the .91 is computed:

In [14]:
edit_distance = 1

1 - edit_distance / (len("Hello1") + len("Hello"))


0.9090909090909091

For more information see: [Levenshtein Distance](https://en.wikipedia.org/wiki/Levenshtein_distance)

## Configuation

To create the necessary configurations for using an Opik Optimizer, you'll need two things:

1. An initial prompt
2. A metric

We're going to start with a pretty bad prompt... so we can optimize it!

In [15]:
from opik_optimizer import ChatPrompt

initial_prompt = ChatPrompt(
    messages=[
        {"role": "system", "content": "Provide an answer to the question"},
        {"role": "user", "content": "{answer}"}
    ]
)

The the two configurations:

In [16]:
def levenshtein_ratio(dataset_item, llm_output):
    metric = LevenshteinRatio()
    return metric.score(reference=dataset_item['answer'], output=llm_output)

As you can see the MetricConfig is composed of our chosen metric. In addition, we need to know what the inputs will be. The inputs here are actually the outputs from the LLM.

We need two inputs for the metric:
1. The output produced by the LLM (uses a special name)
2. The correct answer (provided by the database item "answer")

The TaskConfig defines how to process a prompt. We need the initial prompt, and the inputs and outputs of the dataset.

In this case, we will use the chat_prompt format as our result.

## FewShotBayesianOptimizer

The FewShotBayesianOptimizer name indicates two things:

1. It will produce Chat Prompts, or FewShot examples as described in the slides.
2. Secondly, it describes how it searches for the best set of these FewShot examples.

To use this optimizer, we import it and create an instance, passing in the project name and model parameters:

In [17]:
from opik_optimizer import (
    FewShotBayesianOptimizer,
)

optimizer = FewShotBayesianOptimizer(
    project_name=project_name,
    model="openai/gpt-4o-mini",
    temperature=0.1,
    max_tokens=5000,
)

### Baseline

Before we optimize this prompt ("Provide an answer to the question") let's see what the bare prompt does by itself on the dataset:

In [18]:
score = optimizer.evaluate_prompt(
    prompt=initial_prompt,
    dataset=opik_dataset,
    metric=levenshtein_ratio,
    n_samples=100,
)
score

Evaluation:   0%|          | 0/100 [00:00<?, ?it/s]

╭─ hotpot_300 (100 samples) ─────────────╮
│                                        │
│ Total time:        00:00:23            │
│ Number of samples: 100                 │
│                                        │
│ levenshtein_ratio_metric: 0.0551 (avg) │
│                                        │
╰────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=75578;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=0197406b-33db-78f3-ab35-253c037eb47a&dataset_id=0197044c-d51d-7cff-96c5-38969b6ae708&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

0.05511351451443435

It scored about 16% correct. [I say "percent correct" but because we are using edit distance, that isn't quite accurate. But we can think of it this way.]

Ok, let's optimize that prompt!

In [19]:
result1 = optimizer.optimize_prompt(
    prompt=initial_prompt,
    dataset=opik_dataset,
    metric=levenshtein_ratio,
    n_trials=3,
    n_samples=50
)

╭────────────────────────────────────────────────────────────────────╮
│ ● Running Opik Evaluation - Few-Shot Bayesian Optimizer            │
╰────────────────────────────────────────────────────────────────────╯

> Let's optimize the prompt:

[2025-06-05 15:08:01] ERROR    FewShotBayesian optimization failed: name         ]8;id=634378;file:///Users/jacquesverre/Documents/code/opik/sdks/opik_optimizer/src/opik_optimizer/few_shot_bayesian_optimizer/few_shot_bayesian_optimizer.py\few_shot_bayesian_optimizer.py]8;;\:]8;id=719413;file:///Users/jacquesverre/Documents/code/opik/sdks/opik_optimizer/src/opik_optimizer/few_shot_bayesian_optimizer/few_shot_bayesian_optimizer.py#484\484]8;;\
                               'console' is not defined                                                            
                               ╭────── Traceback (most recent call last) ──────╮                                   
                               │ /Users/jacquesverre/Documents/code/opik/sdks/ │                                   
                               │ opik_optimizer/src/opik_optimizer/few_shot_ba │                                   
                               │ yesian_optimizer/few_shot_bayesian_optimizer. │                                   
                               │ py:429 in optimize_prompt                     │                                   
                               │                                               │                                   
                               │   426 │   │   try:                            │                                   
                               │   427 │   │   │   # Start experiment reportin │                                   
                               │   428 │   │   │   reporting.display_header("F │                                   
                               │       verbose=self.verbose)                   │                                   
                               │ ❱ 429 │   │   │   reporting.display_configura │                                   
                               │   430 │   │   │   │   prompt.formatted_messag │                                   
                               │   431 │   │   │   │   optimizer_config={      │                                   
                               │   432 │   │   │   │   │   "optimizer": self._ │                                   
                               │                                               │                                   
                               │ /Users/jacquesverre/Documents/code/opik/sdks/ │                                   
                               │ opik_optimizer/src/opik_optimizer/reporting_u │                                   
                               │ tils.py:160 in display_configuration          │                                   
                               │                                               │                                   
                               │   157 │   # Panel for Optimizer configuration │                                   
                               │   158 │   rich.print(Text("> Let's optimize t │                                   
                               │   159 │                                       │                                   
                               │ ❱ 160 │   display_messages(messages)          │                                   
                               │   161 │                                       │                                   
                               │   162 │   # Panel for configuration           │                                   
                               │   163 │   rich.print(Text(f"\nUsing {optimize │                                   
                               │                                               │                                   
                               │ /Users/jacquesverre/Documents/code/opik/sdks/ │                                   
                               │ opik_optimizer/src/opik_optimizer/reporting_u │                                   
                               │ tils.py:81 in display_messages   

NameError: name 'console' is not defined

In [ ]:
result1.display()

What did we find? The result is a series of messages:

In [ ]:
result1.details["chat_messages"]

We'll see how we can use those in a few minutes.

## MetaPromptOptimizer

The MetaPromptOptimizer uses a clever idea: have the LLM generate better prompts!

Here is the internal system meta-prompt to have the LLM generate better prompts.

```text
You are an expert prompt engineer. Your task is to improve prompts for any type of task.

Focus on making the prompt more effective by:

1. Being clear and specific about what is expected
2. Providing necessary context and constraints
3. Guiding the model to produce the desired output format
4. Removing ambiguity and unnecessary elements
5. Maintaining conciseness while being complete

Return a JSON array of prompts with the following structure:
{
    "prompts": [
        {
            "prompt": "the improved prompt text",
            "improvement_focus": "what aspect this prompt improves",
            "reasoning": "why this improvement should help"
        }
    ]
}
```

This can work quite well on simpler datasets. It doesn't do so well on HotPot as we will see.

The MetaPromptOptimizer will try a number of rounds to try to find the best prompt.

In [ ]:
from opik_optimizer import (
    MetaPromptOptimizer,
)

optimizer = MetaPromptOptimizer(
    project_name=project_name,
    model="openai/gpt-4o-mini",  # Using gpt-4o-mini for evaluation for speed
    max_rounds=1,  # Number of optimization rounds
    num_prompts_per_round=2,  # Number of prompts to generate per round
    improvement_threshold=0.01,  # Minimum improvement required to continue
    temperature=0.1,  # Lower temperature for more focused responses
    max_completion_tokens=5000,  # Maximum tokens for model completion
    num_threads=1,  # Number of threads for parallel evaluation
    subsample_size=20,  # Fixed subsample size
)


We won't do too many rounds, as this is an impossible problem without tools.

In [ ]:
result2 = optimizer.optimize_prompt(
    dataset=opik_dataset,
    metric=levenshtein_ratio,
    auto_continue=False,
    n_samples=20,  # Explicitly set
    use_subsample=True,  # Force using subsample for evaluation rounds
)

In [ ]:
result2.display()

## MiproOptimizer

MIPRO (Multi-Iteration Prompt Optimization) is an optimizer algorithm that refines both prompts and few-shot examples in a multi-stage LLM program. It works by generating, evaluating, and refining prompts to improve language model performance. MIPRO is a more advanced method than simply "prompt hacking," offering real optimization of LLM workflows.

This sophisticated method optimizes both instructions and examples together. Using Bayesian optimization (like the FewShotBayesianOptimizer), it finds the best combinations of both elements. Through multiple testing rounds, it creates an optimized prompt that pairs effective instructions with relevant examples.

For thi first optimization, we aren't going to give it any tools to work with. Let's see how it works:

In [22]:
from opik_optimizer import MiproOptimizer

optimizer = MiproOptimizer(
    model="openai/gpt-4o-mini",  # LiteLLM or OpenAI name
    project_name=project_name,
    temperature=0.1,
    num_threads=16,
)

In order to use the MiproOptimizer we will need to define a task config

In [23]:

from opik_optimizer import TaskConfig

system_prompt = initial_prompt.formatted_messages[0]["content"]

task_config = TaskConfig(
    instruction_prompt=system_prompt,
    input_dataset_fields=["question"],
    output_dataset_field="answer",
    use_chat_prompt=True,
)


In [24]:
result3 = optimizer.optimize_prompt(
    dataset=opik_dataset,
    metric=levenshtein_ratio,
    task_config=task_config,
    n_samples=50,
    auto="light",
)

[2025-06-05 15:11:34] WARNING  Opik server does not support optimizations. Please upgrade    ]8;id=982841;file:///Users/jacquesverre/Documents/code/opik/sdks/opik_optimizer/src/opik_optimizer/mipro_optimizer/mipro_optimizer.py\mipro_optimizer.py]8;;\:]8;id=552438;file:///Users/jacquesverre/Documents/code/opik/sdks/opik_optimizer/src/opik_optimizer/mipro_optimizer/mipro_optimizer.py#239\239]8;;\
                               opik.                                                                               

                      WARNING  Continuing without Opik optimization tracking.                ]8;id=105458;file:///Users/jacquesverre/Documents/code/opik/sdks/opik_optimizer/src/opik_optimizer/mipro_optimizer/mipro_optimizer.py\mipro_optimizer.py]8;;\:]8;id=484107;file:///Users/jacquesverre/Documents/code/opik/sdks/opik_optimizer/src/opik_optimizer/mipro_optimizer/mipro_optimizer.py#245\245]8;;\


RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 7
minibatch: False
num_candidates: 5
valset size: 40


==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
These will be used as few-shot example candidates for our program and for creating instructions.

Bootstrapping N=5 sets of demonstrations...
Bootstrapping set 1/5
Bootstrapping set 2/5
Bootstrapping set 3/5


 10%|█         | 1/10 [00:00<00:08,  1.12it/s]OPIK: Started logging traces to the "optimize-workshop-2025" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=0197406e-d95f-7681-89ea-45bc8f530ecd&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.
OPIK: Started logging traces to the "Default Project" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=0197406e-decc-7616-b3b1-2ead014b9653&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.
 20%|██        | 2/10 [00:02<00:09,  1.19s/it]OPIK: Started logging traces to the "optimize-workshop-2025" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=0197406e-dece-7dbe-8179-69ace3e87644&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.
OPIK: Started logging traces to the "Default Project" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=0197406e-e1fa-7059-84aa-a8346a0e2466&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.
 3

Bootstrapped 4 full traces after 4 examples for up to 1 rounds, amounting to 4 attempts.
Bootstrapping set 4/5


...

Average Metric: 14.48 / 39 (37.1%):  98%|█████████▊| 39/40 [00:05<00:00,  6.53it/s]

OPIK: Started logging traces to the "optimize-workshop-2025" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=01974070-0a18-7d3c-a7b3-cceb50992114&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.
OPIK: Started logging traces to the "Default Project" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=01974070-12a9-7724-ab8b-e70df792bc34&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.


Average Metric: 15.12 / 40 (37.8%): 100%|██████████| 40/40 [00:05<00:00,  6.80it/s]
Best full score so far! Score: 37.8
Score: 37.8 with parameters ['Predictor 0: Instruction 2', 'Predictor 0: Few-Shot Set 4'].
Scores so far: [0.20173393120947108, 20.21, 20.21, 20.17, 23.09, 19.85, 20.17, 37.8]
Best score so far: 37.8


Returning best identified program with score 37.8!


[2025-06-05 15:12:56] ERROR    Mipro optimization failed: 'function' object has no attribute ]8;id=8859;file:///Users/jacquesverre/Documents/code/opik/sdks/opik_optimizer/src/opik_optimizer/mipro_optimizer/mipro_optimizer.py\mipro_optimizer.py]8;;\:]8;id=598371;file:///Users/jacquesverre/Documents/code/opik/sdks/opik_optimizer/src/opik_optimizer/mipro_optimizer/mipro_optimizer.py#264\264]8;;\
                               'name'                                                                              
                               ╭──────────── Traceback (most recent call last) ────────────╮                       
                               │ /Users/jacquesverre/Documents/code/opik/sdks/opik_optimiz │                       
                               │ er/src/opik_optimizer/mipro_optimizer/mipro_optimizer.py: │                       
                               │ 248 in optimize_prompt                                    │                       
                               │                                                           │                       
                               │   245 │   │   │   logger.warning("Continuing without Opik │                       
                               │   246 │   │                                               │                       
                               │   247 │   │   try:                                        │                       
                               │ ❱ 248 │   │   │   result = self._optimize_prompt(         │                       
                               │   249 │   │   │   │   dataset=dataset,                    │                       
                               │   250 │   │   │   │   metric=metric,                      │                       
                               │   251 │   │   │   │   task_config=task_config,            │                       
                               │                                                           │                       
                               │ /Users/jacquesverre/Documents/code/opik/sdks/opik_optimiz │                       
                               │ er/src/opik_optimizer/mipro_optimizer/mipro_optimizer.py: │                       
                               │ 296 in _optimize_prompt                                   │                       
                               │                                                           │                       
                               │   293 │   │   │   **kwargs,                               │                       
                               │   294 │   │   )                                           │                       
                               │   295 │   │   logger.info("Starting MIPRO compilation..." │                       
                               │ ❱ 296 │   │   result = self.continue_optimize_prompt()    │                       
                               │   297 │   │   logger.info("MIPRO optimization complete.") │                       
                               │   298 │   │   return result                               │                       
                               │   299                                                     │                       
                               │                                                           │                       
                               │ /Users/jacquesverre/Documents/code/opik/sdks/opik_optimiz │                       
                               │ er/src/opik_optimizer/mipro_optimizer/mipro_optimizer.py: │                       
                               │ 455 in continue_optimize_prompt                           │                       
                               │                                                           │                       
                               │   452 │   │   │   }                                       │                       


AttributeError: 'function' object has no attribute 'name'

In [ ]:
result3.display()

In [ ]:
result3.demonstrations

### Agent with Tools

Now we'll try with tools. This will allow multi-prompt optimization.

First, we need a tool. We'll use this one from DSPy:

In [ ]:
# Tools:
import dspy

def search_wikipedia(query: str) -> list[str]:
    """
    This agent is used to search wikipedia. It can retrieve additional details
    about a topic.
    """
    results = dspy.ColBERTv2(url="http://20.102.90.50:2017/wiki17_abstracts")(
        query, k=3
    )
    return [x["text"] for x in results]

Let's test it out on a subject:

In [ ]:
search_wikipedia("Developmental Robotics")

And it is easy to add the tools to the config. Let's go!

In [ ]:
task_config.tools = [search_wikipedia]

result4 = optimizer.optimize_prompt(
    dataset=opik_dataset,
    metric=levenshtein_ratio,
    task_config=task_config,
    n_samples=50,
    auto="light",
)

In [ ]:
result4.display()

In [ ]:
result4.demonstrations

## Using Optimized Prompts

Recall:

1. result1 - FewShotBayesianOptimizer
2. result2 - MetaPromptOptimizer
3. result3 - MiproOptimizer (no tools)
4. result4 - MiproOptimizer (with search_wikipedia)

How can we use the optimized results?

For the first one, recall that the fewshot examples are here:

In [ ]:
result1.details["chat_messages"]

So, once we have those we can do the following:

In [ ]:
from litellm.integrations.opik.opik import OpikLogger
import litellm
opik_logger = OpikLogger()
litellm.callbacks = [opik_logger]

def query(question, chat_messages):
    messages = chat_messages[:-1] # Cut off the last one
    # replace it with question in proper format:
    messages.append({'role': 'user', 'content': '{"question": "%s"}"}' % question})

    response = litellm.completion(
        model="gpt-4o-mini",
        temperature=0.1,
        max_tokens=5000,
        messages=messages,
    )
    return response.choices[0].message.content

In [ ]:
query("When was David Chalmers born?", result1.details["chat_messages"])

In [ ]:
query("What weighs more: a baby elephant or an SUV?", result1.details["chat_messages"])

If it says "elephant" that is not correct!

Let's try that same question with a tool:

In [ ]:
result = result4.details["program"](question="What weighs more: a baby elephant or an SUV?")
result.answer

Well done optimizer!

We'll now head back to the slides to summarize the workshop.

# Resources

1. [Opik Optimizer Workshop Slides](https://bit.ly/opik-optimizer-dsblank-slides)